In [146]:
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

In [148]:
from gensim.models import Phrases
from gensim.models.phrases import Phraser
   
#df = pd.read_csv('data/grill-and-bbq-normalize.csv', encoding='utf-8', header=1, sep='|')
dfrecipe = pd.read_csv('data/recipeset.csv', encoding='utf-8', header=0, sep='|')
dfingre = pd.read_csv('data/ingredients.csv', encoding='utf-8', header=0, sep='|')

df = []
for i in range(len(dfrecipe)):
    j = [dfingre[dfingre['dish_id'] == dfrecipe.dish_id[i]]]
    str = j[0]['ingredient'].values 
    df.append([str[k] for k in range(len(str))])

#recipeSet = [row[5].lower() for row in df.itertuples()]

#bow_ingre = [reci.split(" ") for reci in recipeSet]
bow_ingre = df

phrases = Phrases(bow_ingre, min_count=2, threshold=2)
bigram = Phraser(phrases)
documents = [bigram[j] for j in bow_ingre]

new_bowingre = [doc for doc in documents]
corp_dict = corpora.Dictionary(new_bowingre)
    
corpus = [corp_dict.doc2bow(ingre) for ingre in new_bowingre]

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=3, id2word = corp_dict, passes=20)
print(ldamodel.print_topics(num_topics=3, num_words=4))
print(ldamodel.print_topics(num_topics=3, num_words=5))
print(ldamodel.print_topics(num_topics=3, num_words=6))
print(ldamodel.print_topics(num_topics=3, num_words=7))



[(0, '0.083*"olive oil" + 0.066*"garlic" + 0.051*"lemon" + 0.046*"black pepper"'), (1, '0.071*"garlic" + 0.054*"soy sauce" + 0.040*"ginger" + 0.035*"cilantro"'), (2, '0.062*"garlic" + 0.050*"salt" + 0.047*"black pepper" + 0.045*"onion"')]
[(0, '0.083*"olive oil" + 0.066*"garlic" + 0.051*"lemon" + 0.046*"black pepper" + 0.044*"salt"'), (1, '0.071*"garlic" + 0.054*"soy sauce" + 0.040*"ginger" + 0.035*"cilantro" + 0.028*"pepper"'), (2, '0.062*"garlic" + 0.050*"salt" + 0.047*"black pepper" + 0.045*"onion" + 0.044*"brown sugar"')]
[(0, '0.083*"olive oil" + 0.066*"garlic" + 0.051*"lemon" + 0.046*"black pepper" + 0.044*"salt" + 0.039*"chicken breast"'), (1, '0.071*"garlic" + 0.054*"soy sauce" + 0.040*"ginger" + 0.035*"cilantro" + 0.028*"pepper" + 0.027*"lime juice"'), (2, '0.062*"garlic" + 0.050*"salt" + 0.047*"black pepper" + 0.045*"onion" + 0.044*"brown sugar" + 0.039*"paprika"')]
[(0, '0.083*"olive oil" + 0.066*"garlic" + 0.051*"lemon" + 0.046*"black pepper" + 0.044*"salt" + 0.039*"chicken

TO VERIFY IF RECIPE IS NEW

In [238]:
import numpy as np
s = pd.Series([k for k in bow_ingre])
for k in s:
    ind=[]
    ky = Series(k)
    #This should return True on all ingredients
    #ind=ky.isin(['onion', 'red pepper flakes','pork', 'toast', 'coffee', \
                 #'worcestershire sauce','black pepper' ,'liquid smoke','garlic'])
    #test2
    #ind=ky.isin(['garlic', 'salt','black pepper', 'onion', 'brown sugar','paprika','pork'])
    #test3
    ind=ky.isin(['garlic', 'soy sauce','ginger', 'cilantro', 'pepper', 'lime juice','salt'])
    if (np.sum(ind)==len(ind)):
        print('Found the recipe with ' + ky)



0    Found the recipe with salt
dtype: object
0    Found the recipe with garlic
dtype: object
0     Found the recipe with soy sauce
1    Found the recipe with lime juice
dtype: object
0    Found the recipe with garlic
dtype: object
0    Found the recipe with ginger
dtype: object
0    Found the recipe with garlic
dtype: object
0       Found the recipe with garlic
1    Found the recipe with soy sauce
dtype: object
